In [2]:
from src.json_models.src.model_generator import ModelGenerator

In [15]:
gen = ModelGenerator(json_path="/home/andrewheschl/PycharmProjects/ClassificationPipeline/models/mobilenets/polys/xmodules/xmodule_poly_1_3_3_5.json")

In [16]:
model = gen.get_model()
print(gen.get_log_kwargs())

{'backbone': 'Mobilenetv2', 'conv': 'Poly with X[3, 5]', 'order': '1, 3', 'mode': 'sum'}


In [17]:
import torch
data = torch.ones((1, 3, 320, 320))
out = model(data)
out.shape

torch.Size([1, 7])

In [18]:
model

ModelBuilder(
  (self_modules): ModuleList(
    (0): MobileNetV2(
      (stem_conv): Sequential(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (layers): Sequential(
        (0): InvertedBlock(
          (layers): Sequential(
            (0): PolyWrapper(
              (branches): ModuleList(
                (0-1): 2 x PolyBlock(
                  (conv): XModule(
                    (branches): ModuleList(
                      (0): Sequential(
                        (0): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1))
                        (1): Conv2d(16, 16, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1), groups=16)
                        (2): Conv2d(16, 16, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0), groups=16)
                      )
                      (1): Sequential(
             